# ETL Pipeline Preparation

### 0. Import libraries and load datasets

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
dfmsg = pd.read_csv('./datasets/messages.csv')
dfmsg.sample(2, random_state= 0)

,id,message,original,genre
8619,9635,hello! I have the ability to work at the world...,salut. mwen gen kapasite pou'm travay nan recr...,direct
26039,30033,"Like most families in the delta, Win and her h...",NaN,news


### 1. Remove duplicates.

In [3]:
# There are duplicates in the dataset
dfmsg.shape, dfmsg.drop_duplicates().shape

((26248, 4), (26180, 4))

In [4]:
# Remove the duplicate rows
dfmsg = dfmsg.drop_duplicates()

In [5]:
# load categories dataset
dfcats = pd.read_csv('./datasets/categories.csv')
dfcats.sample(2, random_state= 0)

,id,categories
8619,9635,related-1;request-1;offer-0;aid_related-0;medi...
26039,30033,related-0;request-0;offer-0;aid_related-0;medi...


In [6]:
# Also, here, there are duplicates
dfcats.shape, dfcats.drop_duplicates().shape

((26248, 2), (26216, 2))

In [7]:
# Remove duplicate rows
dfcats = dfcats.drop_duplicates()

In [8]:
cats_counts = dfcats['id'].value_counts()
cats_counts[cats_counts > 1].shape

(36,)

In [9]:
# The followings appear twice in the categories dataset
cats_counts[cats_counts > 1].index

Int64Index([15760, 12416, 12420, 24247,  3882, 11503,  6492, 14135,  3250,
            27768,  6393,  5776, 13914, 18925,  7945,  1652, 16245, 10286,
            17553, 19142,   862, 19687,  4956,  7747, 25512,  6515, 22858,
             5153, 28462, 19003, 17385, 29022, 24347,   202,  8190,  6687],
           dtype='int64')

#### Observe the disagreements for a couple of messages:

In [10]:
dfx = dfcats.loc[dfcats['id'] == 17553]['categories'].str.split(';', expand= True).T
dfx.loc[dfx[14836] != dfx[14837]]

,14836,14837
1,request-1,request-0
3,aid_related-1,aid_related-0
4,medical_help-1,medical_help-0
20,transport-1,transport-0
28,weather_related-1,weather_related-0
29,floods-1,floods-0
34,other_weather-1,other_weather-0
35,direct_report-1,direct_report-0


In [11]:
dfx = dfcats.loc[dfcats['id'] == 19003]['categories'].str.split(';', expand= True).T
dfx.loc[dfx[16155] != dfx[16156]]

,16155,16156
1,request-1,request-0
33,cold-0,cold-1


In [12]:
dfmsg.loc[dfmsg['id'] == 19003, 'message'].values[0]

"Before we take your general questions, I'd like to make a few opening remarks concerning ISAF support to the Afghan people during these frigid winter months. After the earthquake, huge boulders blocked off the road to large vehicles. Most years, floods wreak havoc in the state, leaving a trail of destruction and killing hundreds of people. We have heard of crocodile sightings."

### Note:
There are 36 messages that have 2 category groups registered for them. A possible reason could be that multiple people have labeled those messages. This in fact is a sign of the difficulty of the task.

I could handle this issue in multiple ways.   

__1) Remove the problematic messages (that have opposing values for some of the 36 categories)__   
This will lead to loss of valuable information. Not a good strategy.

__2) Read the messages, and try to resolve the problem.__   
This could be a viable solution, considering that we only have 36 such messages; but it's tedious, even for such small cases, and perhaps expert domain knowledge would be necessary.

__3) Accept both categories as is.__ 
This may seem strange, but I believe that few such misbehaved message categories could lead to roubustness of the model, and lead to better performance for forecasting future unseen data. After all, the messages are not easy to categorize. But I'll perhaps remove such cases from the test set. Because when predicting, we want to be sure, 0 or 1 for each category. 



### 2. Merge datasets

In [13]:
df = dfmsg.merge(dfcats, how= 'left', on= 'id')
df.sample(10, random_state= 0)

,id,message,original,genre,categories
13763,16395,Schools have also been closed due to the incle...,NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
21716,25209,UNICEF has already sent supplies to affected a...,NaN,news,related-1;request-0;offer-1;aid_related-1;medi...
16002,18859,Factors other than the state of tropical Pacif...,NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
3745,4294,Mayissade has a lot of victims coming from Por...,( Mayisad gen anpil moun ki te viktim potoprin...,direct,related-1;request-0;offer-0;aid_related-1;medi...
12096,13992,We are 17 people. Our house has been immersed(...,Hum 17 Log hain Hamara Ghar Dobh Gaya hai Aur ...,direct,related-1;request-0;offer-0;aid_related-1;medi...
15578,18384,"However, these aircraft cannot fly in poor wea...",NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
24931,28804,As the Southern Hemisphere enters its influenz...,NaN,news,related-1;request-0;offer-0;aid_related-1;medi...
17000,19970,No rainfall was recorded in the main river bas...,NaN,news,related-1;request-0;offer-0;aid_related-0;medi...
2164,2485,How will you do for the unable Ethe poor?,Ki mwayen nap fe pou moun ki pa gen posiblite yo?,direct,related-0;request-0;offer-0;aid_related-0;medi...
23726,27459,More than 40 people who had spent a night in c...,NaN,news,related-1;request-0;offer-0;aid_related-1;medi...


In [14]:
# The dimensions make sense
## There are 36 extra cases in categories (dfcats) as mentioned earlier.
## This will carry over when merging the two datasets (messages & categories)
dfmsg.shape, dfcats.shape, df.shape

((26180, 4), (26216, 2), (26216, 5))

### 3. Split `categories` into separate category columns

In [15]:
# Split the `categories` column into 36 different categories:
dflabels = df['categories'].str.split(';', expand= True)
# Show first 3 rows, and first 12 categories
dflabels.loc[:3, :11]

,0,1,2,3,4,5,6,7,8,9,10,11
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,water-0,food-0


In [16]:
# Get the category names
labels = [x.split('-')[0] for x in dflabels.loc[0, ].values.tolist()]
labels

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [17]:
# Set the column names to their proper labels:
dflabels.columns = labels
dflabels.head(6).T

,0,1,2,3,4,5
related,related-1,related-1,related-1,related-1,related-1,related-0
request,request-0,request-0,request-0,request-1,request-0,request-0
offer,offer-0,offer-0,offer-0,offer-0,offer-0,offer-0
aid_related,aid_related-0,aid_related-1,aid_related-0,aid_related-1,aid_related-0,aid_related-0
medical_help,medical_help-0,medical_help-0,medical_help-0,medical_help-0,medical_help-0,medical_help-0
medical_products,medical_products-0,medical_products-0,medical_products-0,medical_products-1,medical_products-0,medical_products-0
search_and_rescue,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0,search_and_rescue-0
security,security-0,security-0,security-0,security-0,security-0,security-0
military,military-0,military-0,military-0,military-0,military-0,military-0
child_alone,child_alone-0,child_alone-0,child_alone-0,child_alone-0,child_alone-0,child_alone-0


In [18]:
# Convert the values for each category/label to binary 
## 1: if belongs to that category, 0: otherwise
for col in dflabels.columns:
    dflabels[col] = dflabels[col].str.get(-1)

In [19]:
dflabels.iloc[:5, :14]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
df.shape, dflabels.shape

((26216, 5), (26216, 36))

### 4. Replace `categories` column in `df` with new category columns

In [21]:
# Merge these newly created categories with the original dataset,
## Since the `categories` column - from which the labels/categories- have been created
## is not needed anymore, drop it from the dataset.
df = pd.concat([df, dflabels.reindex(df.index)], axis=1).drop(['categories'], axis= 1)

df.iloc[:4, :10]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1


### 5. Convert category values to just numbers 0 or 1

In [22]:
# Convert the category/label columns to numeric-binary
df[labels] = df[labels].apply(pd.to_numeric, axis = 1)

In [23]:
# Make sure conversion is correct
df.dtypes

id                         int64
message                   object
original                  object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_center

In [24]:
df['related'].value_counts(dropna= False)

1    19906
0     6122
2      188
Name: related, dtype: int64

In [25]:
df['cold'].value_counts(dropna= False)

0    25686
1      530
Name: cold, dtype: int64

In [26]:
for col in labels:
    print(col, '====> ', set(df[col]))

related ====>  {0, 1, 2}
request ====>  {0, 1}
offer ====>  {0, 1}
aid_related ====>  {0, 1}
medical_help ====>  {0, 1}
medical_products ====>  {0, 1}
search_and_rescue ====>  {0, 1}
security ====>  {0, 1}
military ====>  {0, 1}
child_alone ====>  {0}
water ====>  {0, 1}
food ====>  {0, 1}
shelter ====>  {0, 1}
clothing ====>  {0, 1}
money ====>  {0, 1}
missing_people ====>  {0, 1}
refugees ====>  {0, 1}
death ====>  {0, 1}
other_aid ====>  {0, 1}
infrastructure_related ====>  {0, 1}
transport ====>  {0, 1}
buildings ====>  {0, 1}
electricity ====>  {0, 1}
tools ====>  {0, 1}
hospitals ====>  {0, 1}
shops ====>  {0, 1}
aid_centers ====>  {0, 1}
other_infrastructure ====>  {0, 1}
weather_related ====>  {0, 1}
floods ====>  {0, 1}
storm ====>  {0, 1}
fire ====>  {0, 1}
earthquake ====>  {0, 1}
cold ====>  {0, 1}
other_weather ====>  {0, 1}
direct_report ====>  {0, 1}


The `related` label has some values as `2` which is weird. Also, at least one column - `child_alone` - is `zero-variance`, i.e., has one single value.

In [27]:
df.loc[df['related'] == 2].head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
dfcats.loc[dfcats['id'].isin(df.loc[df['related'] == 2].head(3)['id'])]

,id,categories
117,146,related-2;request-0;offer-0;aid_related-0;medi...
219,263,related-2;request-0;offer-0;aid_related-0;medi...
305,373,related-2;request-0;offer-0;aid_related-0;medi...


So, it seems there is no but in the data cleaning; I'll think about what the value `2` means later when doing the modeling.

### 6. Save the clean dataset into an sqlite database

In [29]:
engine = create_engine('sqlite:///DisasterResponseDB.db')
df.to_sql('DisasterResponse', engine, index=False)